In [8]:
from core.utils import Tibanna
from core import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)

file_not_found = 0
deleted = 0
success = 0
patch_problem = 0
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
files_resp = ff_utils.get_metadata('files', connection=ff)['@graph']
print len(files_resp), 'files'
no_size = 0
for a_file in files_resp:
    if a_file['status'] == 'deleted':
        deleted += 1
        continue
    # check if there is  no filesize
    if not a_file.get('file_size'):
        # decide on the bucket
        if 'FileProcessed' in a_file['@type']:
            bucket = tibanna.s3.outfile_bucket
        else:
            bucket = tibanna.s3.raw_file_bucket
            
        # check if file is in s3
        head_info = tibanna.s3.does_key_exist(a_file['upload_key'], bucket)
        if not head_info:
            print a_file['@id'], a_file['status']
            file_not_found += 1
            continue
        file_size = head_info['ContentLength']

        patch_data = {'file_size': file_size}
        try:
            ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] ,connection=ff)
            success += 1
        except Exception as e:
            print e
            print
            patch_problem += 1

print file_not_found, 'files missing from s3'
print patch_problem, 'files had patch problems'
print deleted, 'deleted files skipped'
print success, 'total files patched'

4131 files
An error occurred (404) when calling the HeadObject operation: Not Found
/files-fasta/4DNFI6K4IMBU/ uploading
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFN8ICRHEN/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFQOZAY8O8/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFMH6Z2ZD5/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFHFYGN09R/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFSYWQ59JI/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFEUAGRP16/ uploaded
An error occurred (404) when calling the HeadObject operation: Not Found
/files-processed/4DNFIN232ZZZ/ uploading
An error occurred (404) when calling the HeadObject operation: Not Found
/files-process